In [57]:
import requests
import json
import time
import pandas as pd
import numpy as np

In [61]:
API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'
def single_geocode(address):
    print('fetching %s' % address)
    res = requests.get(API_URL, params={
            'address' : address,
            'sensor' : 'false',
        })
    time.sleep(1)
    val = json.loads(res.content.decode('utf8'))
    print(val)
    return val
    
def bulk_geocode(addresses):
    return {address : single_geocode(address) for address in addresses}

In [3]:
returns = pd.read_excel('results/israel/knesset-city.xlsx')

In [4]:
returns.Name.head()

0     ABU JUWEIID, ISRAEL
1       ABU GHOSH, ISRAEL
2       ABU SINAN, ISRAEL
3       Abu Abdun, ISRAEL
4    Abu Kurejnat, Israel
Name: Name, dtype: object

In [ ]:
geodat = bulk_geocode(returns.Name)

fetching ABU JUWEIID, ISRAEL
{'results': [], 'status': 'ZERO_RESULTS'}
fetching ABU GHOSH, ISRAEL
{'results': [{'types': ['locality', 'political'], 'address_components': [{'types': ['locality', 'political'], 'long_name': 'Abu Ghosh', 'short_name': 'Abu Ghosh'}, {'types': ['administrative_area_level_1', 'political'], 'long_name': 'Jerusalem District', 'short_name': 'Jerusalem District'}, {'types': ['country', 'political'], 'long_name': 'Israel', 'short_name': 'IL'}], 'geometry': {'bounds': {'southwest': {'lat': 31.7991071, 'lng': 35.094697}, 'northeast': {'lat': 31.8138969, 'lng': 35.122427}}, 'location_type': 'APPROXIMATE', 'location': {'lat': 31.806293, 'lng': 35.109193}, 'viewport': {'southwest': {'lat': 31.7991071, 'lng': 35.094697}, 'northeast': {'lat': 31.8138969, 'lng': 35.122427}}}, 'place_id': 'ChIJ40bxIN_QAhUR7azWMEhNx4c', 'formatted_address': 'Abu Ghosh, Israel'}], 'status': 'OK'}
fetching ABU SINAN, ISRAEL
{'results': [{'types': ['locality', 'political'], 'address_components

In [60]:
geodat

{'TURAN, ISRAEL': {'error_message': 'You have exceeded your rate-limit for this API.',
  'results': [],
  'status': 'OVER_QUERY_LIMIT'},
 'AHISAMAKH, ISRAEL': {'error_message': 'You have exceeded your rate-limit for this API.',
  'results': [],
  'status': 'OVER_QUERY_LIMIT'},
 'SEDEROT, ISRAEL': {'error_message': 'You have exceeded your rate-limit for this API.',
  'results': [],
  'status': 'OVER_QUERY_LIMIT'},
 'SALLAMA, ISRAEL': {'error_message': 'You have exceeded your rate-limit for this API.',
  'results': [],
  'status': 'OVER_QUERY_LIMIT'},
 'BAREQET, ISRAEL': {'error_message': 'You have exceeded your rate-limit for this API.',
  'results': [],
  'status': 'OVER_QUERY_LIMIT'},
 'TAYIBE, ISRAEL': {'error_message': 'You have exceeded your rate-limit for this API.',
  'results': [],
  'status': 'OVER_QUERY_LIMIT'},
 'TALMEI YAFE, ISRAEL': {'error_message': 'You have exceeded your rate-limit for this API.',
  'results': [],
  'status': 'OVER_QUERY_LIMIT'},
 'EN YAAQOV, ISRAEL': {'

In [49]:
def get_adm(results):
    if len(results) == 0:
        return None
#     print(results[0])
    national = [area for area in results[0]['address_components'] if len(area['types']) > 0 and area['types'][0] == 'country']
    if len(national) == 0 or national[0]['short_name'] != 'IL':
        return None
    areas = [area for area in results[0]['address_components'] if len(area['types']) > 0 and area['types'][0] == 'administrative_area_level_1']
    if len(areas) == 0:
        return None
    return areas[0]['short_name']
def get_adm_from_name(name):
    if not name in geodat.keys():
        return None
    return get_adm(geodat[name]['results'])

In [56]:
districts = [get_adm_from_name(n) for n in returns['Name']]

In [54]:
len([d for d in districts if d is None])

986

In [55]:
len(districts)

1195